# 딥러닝 관련 논문
- https://arxiv.org

- 아래는 딥러닝 관련해서만 논문이 올라와있음
- https://paperswithcode.com/ 


# Object Detection 모델

- <span style='font-size:1.3em;font-weight:bold'>Object Detection = Localization + Classification</span>
    - **Object Detection** 은 이미지에 존재하는 Object(물체)들을 Bounding Box를 이용해 그 위치를 찾아내고(Localization) class를 분류(Classification)하는 작업이다.   
    - Deep learning 을 이용한 Object Detection 모델들은  One stage detector와 Two stage detector 두가지 방식이 있다.

## One stage vs Two Stage detector
- **One stage Detector**
    - Localization과 classification을 하나의 네트워크에서 처리한다. 
- **Two stage Detector**
    - Localization과 classification을 처리하는 모델을 따로 만들어서 각각 순차적으로 처리한다. 
    - 정확도가 높은 대신 속도가 느리다.  느리다는 단점 때문에 Real-Time Detection이 안된다.

![image.png](attachment:image.png)
<center><font size=3>Object Detection in 20 Years: A Survey:https://arxiv.org/pdf/1905.05055.pdf</font></center>

# Single Shot Multibox Detector - SSD

## 개요
- YOLO 이후 나온 One stage 계열의 알고리즘
- Multibox Detector 개념을 적용해서 YOLO이상의 속도를 내면서도 detection 성능을 높였다.
- 논문: https://arxiv.org/abs/1512.02325

## SSD 특징
- One stage detection으로 정확도와 속도를 높임.
- YOLO는 속도 측면에서 당시 SOTA 모델인 Faster R-CNN이 7FPS이었던 것을 45FPS까지 끌어올렸으나 정확도는 떨어졌다. 또 작은 물체를 잘 감지 못하는 단점도 있었다. 이런 문제를 해결하려는 시도에서 출발한 것이 SSD이다.
    - Pascal VOC 2007 테스트 데이터 기준으로
        - Faster R-CNN: 7 FPS, 73.2% mAP
        - YOLO: 45 FPS, 63.4% mAP
        - SSD: 59 FPS, 74.3% mAP
    
- SSD 이후 one stage detection 알고리즘들이 ssd의 구조를 기반으로 성능을 높이는데 집중함.

![image-3.png](attachment:image-3.png)

### Single Shot MultiBox Detector
- Single Shot Detector
    - One stage detection으로 input이 들어가 detection 하는 전과정을 하나의 Network가 처리한다.
    - 이것을 통해 빠른 속도의 검출이 가능하다.

- MultiBox 
    - Multi-scale feature maps for detection
    - Convolution을 통과하며 나오는 중간단계의 다양한 크기의 feature map들을 이용해 detection 하여  
    큰 물체와 작은 물체를 잘 detection하도록 한다.
    - 이것을 통해 Detection의 정확도를 높였다.

## Network(Model 구조) 와 Multi Scale Feature Maps for Detection

![image.png](attachment:image.png)

In [1]:
#SSD
"""
화살표가 있는 레이어는 detection을 함 >> 중간 feature map에서 찾는것을 계속함  >> 결과를 여러번 추출 
최종적으로 추론한게 8732개 정도 나오게됨
최종결과 4*(classes+4)
한 박스 내에서 4개를 찾겠다는것 (3x3x4x(classes+4))

위의 그림내의 중간 박스는 feature map임

백본은 맨 앞의 38x38, 그 다다음부터 6개는 보조네트워크
"""


#yolo
#98개씩 만드는 것은 객체가 몇개인지 몰라서 최대 98개를 찾겠다는 의미임
#따라서 98개 정도의 예측박스가 나오고 non maximum suppresion으로 실제 객체라고 예상되는 박스만 남김



'\n화살표가 있는 레이어는 detection을 함 >> 중간 feature map에서 찾는것을 계속함  >> 결과를 여러번 추출 \n최종적으로 추론한게 8732개 정도 나오게됨\n최종결과 4*(classes+4)\n한 박스 내에서 4개를 찾겠다는것 (3x3x4x(classes+4))\n\n위의 그림내의 중간 박스는 feature map임\n\n백본은 맨 앞의 38x38, 그 다다음부터 6개는 보조네트워크\n'

- Input image로 300 X 300 이미지를 사용 (SSD 300; SSD 512는 512 x 512 이미지사용)
- 512가 성능이 더 좋음(보통 해상도가 더 높은게 성능이 좋음 대신 메모리를 많이 차지함)
- 모델 뒤에 붙는 숫자는 이미지 크기

- Base network(백본 network)로 VGG-16(conv5_3까지만 사용)을 사용.
    - Imagenet 데이터셋으로 pretrain.
    - 38 x 38 x 512 Feature map을 추출한다.

# object detection part

- 백본에서 5번 지금 레이어에서 1번 총 6번
- 전체 Network에서 6개의 Convolution Layer(보조네트워크)를 사용해 Feature map을 추출하고 각 Feature map마다 Object Detection을 실행.

- 각 Feature map을 3 x 3 x **N**의 convolution layer를 통과 시켜 x,y,w,h 의 bounding box 위치와 클래스별 확률(Softmax)을 검출한다.
    - **N** : Default box개수 * (class별 확률 + boundingbox위치)
        - <a href='#Default-Boxes-Generating'>Default box</a>개수는 4, 또는 6
        - class별 확률: Pascal VOC dataset 사용기준 21(20 + 1). 20개 class, 1개 배경일 확률.
        - Boundingbox 위치: 4 (x, y, w, h)
        - 배경일 확률은 confidence score가 됨 

- 모든 레이어를 통과하면 각 feature map들을 이용해 검출된 결과를 취합한 뒤 Non-Maximum Suppression을 이용해 최종 결과를 출력한다.   

### Multi Scale Feature Maps for Detection 흐름

- 다양한 size의 Feature map으로부터 객체를 검출하므로 다양한 크기의 object에 대한 검출 정확도가 높아진다.
    - Yolo는 7x7 하나의 grid에서 검출하지만 SSD는 전체 이미지를 38x38, 19x19, 10x10, 5x5, 3x3, 1x1의 그리드를 가진 Feature map들에서 물체를 검출한다.
    - Bottom쪽 feature map에서는 작은 물체를 검출하고 top으로 갈 수록 큰 물체에 대한 검출이 이뤄진다.   
![image.png](attachment:image.png)

In [ ]:
"""
base network=backbone network

38x38x4는 38x38범위 내의 박스 4개의 degault box(=anchor box)를 찾음
위의 네트워크 output을 전부 더해서 8732개

픽셀(=셀)

"""

## Default Boxes Generating
- Cell(Feature map의 pixcel) 별로 비율과 크기가 다른 여러개의 Box들을 먼저 설정해놓고 그것을 이용해 Bounding Box를 추론한다.
    - 이 미리 정의된 box들을 Default Box라고 한다. 다른 모델에서는 Anchor라고 한다.
- Feature map의 grid마다 N개(4 또는 6개)의 Default Box를 사용한다. 
- Feature map 마다 다른 크기(scale)의 다양한 종횡비(aspect ratio)의 default box를 생성해 물체를 검출.

In [ ]:
"""
찾고자 하는 객체의 anchor box 비율을 미리 잡아둬서 정답이라고 예측하는 박스
디폴트 박스를 얼마나 움직이면 정답에 가까워지는지 ground truth bounding box간의 차이 (좌표(x,y,w,h)의 offset(얼마나 떨어져 있는지))를 weight들이 학습

물체가 있을거라고 예측하는 디폴트 박스라는 것을 얼마나 움직였을 때 정답일거라고 좌표를 학습시키는 것임
미리 박스를 쳐놓고 얼마만큼 움직였을 때 해당 객체와 얼마나 일치하는지를 파악하는 원리
"""

> - scale : botton layer부터 추출한 feature map에 적용시킬 default box의 scale    
>     - input image의 size기준으로 각각 **0.2, 0.34, 0.48, 0.62, 0.76, 0.9** 크기로 생성
> - 4개일때 width, height 비율
>     - 1:1, 1:2, 2:1, 1:1(더 큰 정사각형)
> - 6개일때 width, height 비율
>     - 1:1, 1:2, 1:3, 2:1, 3:1, 1:1(더 큰 정사각형)

![image.png](attachment:image.png)

## Training
- SSD는 Ground Truth Bounding Box와의 IoU가 설정한 Threshold(논문은 0.5)인 예측 Bounding Box(default box)에 대해서만 학습한다.
    - 대부분의 모델들은 IoU가 가장 큰 Bounding Box만 사용하는데 SSD는 특정 threshold보다 큰 것들은 모두 선택해 학습 하여 모델이 높은 정확도를 가진 상자들을 많이 학습하게 하므로써 모델의 학습결과를 향상시킬 수 있었다.


### Hard negative mining
 - 예측한 bounding box(default box)들은 맞는 것(positive samples)보다 틀린(Negative samples)것의 양이 훨씬 많다. (이미지 내에 object는 몇개 안되고 대부분이 배경이다.) 이 상태에서 train을 하게 되면 불균형 문제로 제대로 학습이 안되는 문제가 발생한다. 그래서 **추출된 default box의 classification loss(confidence loss)를 내림차순으로 정렬한 뒤 그 중 positive의 3배 정도의 수량만 선택해서 train 시킨다.**(클수록 오차가 큰 것임). 이런 방식을 hard negative mining 이라고 한다. 
- positive sample: ground truth와 IoU가 지정한 threshold **이상인** default box
- negative sample: ground truth와 IoU가 지정한 threshold **미만인** default box
- positive : negative = 1 : 3 비율

In [ ]:
"""
8700개 중에서 GT bbox IOU 계산해서 0.5이상이면 positive 
                                   0.5이하이면 negative 
                                   불균형 데이터는 학습이 잘 안됨 (negative가 훨씬 많거나 positive가 훨씬 많은 경우는 학습이 잘안됨)
                                   
"""

In [ ]:
"""
bounding box 
L1 loss(MAE) = |y_i-y_hat|

L2 loss(MSE) = (y_i - y_hat)^2

특이하게 회귀문제는 mse로 안함
여기선 
"""

### Loss Function
- multi-task loss
    - classification 예측 결과에 대한 오차로 cross entropy를 사용한다.
    - bounding box의 좌표 예측에 대한 오차로 $smooth_{L1} loss$를 사용한다. 
    - 이 두개의 오차를 합쳐 최종 loss를 구한다.

![image.png](attachment:image.png)

- N: Ground Truth와 IoU가 threshold 보다 크게 나온 default box의 개수. N이 0이면 loss를 0으로 설정.
- x: 실제 class
- c: 예측한 class
- l: 예측한 bounding box
- g: 실제 bounding box
- $\large L_{conf}(x,c)$: classification 오차. Cross Entropy 사용
- $\large\alpha L_{loc}(x,l,g)$: Localization - Bounding Box 예측 오차

> - $\large smooth_{L1} loss$
>    - 실제값과 예측값의 L1 Norm (|y-$\hat{y}$|)를 구한다. 결과 값이 1보다 작으면 제곱한 값에 0.5곱한 값(절반)을 1이상이면 그 값에서 0.5 뺀 값을 반환한다.
> ![image-4.png](attachment:image-4.png)

In [ ]:
"""
negative가 많아서 다 더하면 오차가 크게나오므로 -0.5를 빼거나

오차가 1이하라는것은 IOU가 1에 가깝다는 것(=ground truth와 크게 다르지 않다는 것) 따라서 loss를 작게해주기 위해 0.5를 곱함
오차를 빨리 학습되도록 0에 가깝게 만들어버림
 
오차가 크면(otherwise) 0.5를 뺌

"""